# RAG ChatBot 

## Importing necessary libraries

In [1]:

import os
import textwrap
from pathlib import Path

from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

## Parsing the pdf document

In [2]:
#instruction to fine-tune the parser
instruction = """The provided document is Meta First Quarter 2024 Results.
This form provides detailed financial information about the company's performance for a specific quarter.
It includes unaudited financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""     

#setting up LLamaParse to parse through the pdf document
parser = LlamaParse(                                     
    api_key="<your_api_key>", #Llamacloud api ley
    result_type="markdown",   #convert pdf to markdown
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("data/meta-earnings.pdf")

Started parsing the file under job_id bd547311-b4f0-44b7-8f89-a9369ce1da5e


In [3]:
parsed_doc = llama_parse_documents[0]
Markdown(parsed_doc.text[0:4096])

#

# Meta First Quarter 2024 Results

# Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

# First Quarter 2024 Financial Highlights

| |Three Months Ended March 31, 2024|Three Months Ended March 31, 2023|% Change|
|---|---|---|---|
|Revenue|$36,455|$28,645|27%|
|Costs and expenses|$22,637|$21,418|6%|
|Income from operations|$13,818|$7,227|91%|
|Operating margin|38%|25%| |
|Provision for income taxes|$1,814|$1,598|14%|
|Effective tax rate|13%|22%| |
|Net income|$12,369|$5,709|117%|
|Diluted earnings per share (EPS)|$4.71|$2.20|114%|

# First Quarter 2024 Operational and Other Financial Highlights

- Family daily active people (DAP) – DAP was 3.24 billion on average for March 2024, an increase of 7% year-over-year.
- Ad impressions – Ad impressions delivered across our Family of Apps increased by 20% year-over-year.
- Average price per ad – Average price per ad increased by 6% year-over-year.
- Revenue – Total revenue and revenue on a constant currency basis were $36.46 billion and $36.35 billion, respectively, both of which increased by 27% year-over-year.
- Costs and expenses – Total costs and expenses were $22.64 billion, an increase of 6% year-over-year.
- Capital expenditures – Capital expenditures, including principal payments on finance leases, were $6.72 billion.
- Capital return program – Share repurchases were $14.64 billion of our Class A common stock and dividends payments were $1.27 billion.
- Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were $58.12 billion as of March 31, 2024. Free cash flow was $12.53 billion.
- Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.
---
#

# Meta First Quarter 2024 Results

# CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growth, based on current exchange rates.

We expect full-year 2024 total expenses to be in the range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and our investments to further scale our ecosystem.

We anticipate our full-year 2024 capital expenditures will be in the range of $35-40 billion, increased from our prior range of $30-37 billion as we continue to accelerate our infrastructure investments to support our artificial intelligence (AI) roadmap. While we are not providing guidance for years beyond 2024, we expect capital expenditures will continue to increase next year as we invest aggressively to support our ambitious AI research and product development efforts.

Absent any changes to our tax landscape, we expect our full-year 2024 tax rate to be in the mid-teens.

In addition, we continue to monitor an active regulatory landscape, including the increasing legal and regulatory headwinds in the EU and the U.S. that could significantly impact our business and our financial results.

Q1 was a good start to the year. We're seeing strong momentum within our Family of Apps and are making important progress on our longer-term AI and Reality Labs initiatives that have the potential to transform the way people interact with our services over the coming years.
---
#

# Meta First Quarter 2024 Results

# Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can 

## Converting the 

In [7]:
#create a markdown file to store parsed_doc
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

In [10]:
#Load the markdown file
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()


In [11]:
#split the data into smaller 'chunks' for efficient processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)
     

26

In [12]:
print(docs[0].page_content)

Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

First Quarter 2024 Financial Highlights

Three Months Ended March 31, 2024 Three Months Ended March 31, 2023 % Change Revenue $36,455 million $28,645 million 27% Costs and expenses $22,637 million $21,418 million 6% Income from operations $13,818 million $7,227 million 91% Operating margin 38% 25% Provision for income taxes $1,814 million $1,598 million 14% Effective tax rate 13% 22% Net income $12,369 million $5,709 million 117% Diluted earnings per share (EPS) $4.71 $2.20 114%

In [13]:
#get the embeddings
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")    

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
#Use the Qdrant vector similarity search engine
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path="./db",
    collection_name="document_embeddings",
)
     

In [18]:

query = "What is the most important innovation from Meta?"
similar_docs = qdrant.similarity_search_with_score(query)

In [19]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

score: 0.6206513248343075
--------------------------------------------------------------------------------

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

score: 0.6206513248343075
--------------------------------------------------------------------------------

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the

In [20]:

%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 240 ms, sys: 7.9 ms, total: 248 ms
Wall time: 75.6 ms


In [21]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

id: 5fef76b0d6ea45c482545ff753e15d53

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

--------------------------------------------------------------------------------

id: e909fc73499f427c87ba9ef5b0fe2460

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

--------------------------------------------------------------------------------

id: 66ca6472da7544b0aa885b45d097dc6c

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the

In [14]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [15]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

CPU times: user 3.89 s, sys: 236 ms, total: 4.13 s
Wall time: 981 ms


3

In [16]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: 5fef76b0d6ea45c482545ff753e15d53

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

score: 0.5479341745376587
--------------------------------------------------------------------------------

id: 7fe0585116624050abe0f05f4d17935f

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year,

score: 0.01797989383339882
--------------------------------------------------------------------------------

id: 1686512edbba494593fa570cc0f98c17

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (N

In [22]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192",groq_api_key="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")

In [23]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [24]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [27]:

%%time
response = qa.invoke("What is the most significant innovation from Meta?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the world. Now, Meta is moving beyond 2D screens toward immersive experiences like augmented and virtual reality to help build the next evolution in social technology.

Contacts

Investors:

Kenneth Dorell

investor@meta.com / investor.fb.com

Press:

Ryan Moore

press@meta.com / about.fb.com/news/

Meta First Quarter 2024 Results

Meta First Quarter 2024 Results

This press release contains forward-looking statements regarding Meta's future business plans and expecta

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
CPU times: user 3.93 s, sys: 428 ms, total: 4.36 s
Wall time: 2.61 s


In [28]:
print_response(response)

Based on the provided information, the most significant innovation from Meta is the new version of
Meta AI with Llama 3, which is mentioned in the quote from Mark Zuckerberg, Meta founder and CEO.
This innovation is considered a step towards building the world's leading AI.

Additionally, it's worth noting that Meta is also working on building the metaverse, which is
mentioned in the same quote. However, the exact details of this innovation are not provided in the
given information.


In [29]:
Markdown(response["result"])

Based on the provided information, the most significant innovation from Meta is the new version of Meta AI with Llama 3, which is mentioned in the quote from Mark Zuckerberg, Meta founder and CEO. This innovation is considered a step towards building the world's leading AI.

Additionally, it's worth noting that Meta is also working on building the metaverse, which is mentioned in the same quote. However, the exact details of this innovation are not provided in the given information.

In [30]:
%%time
response = qa.invoke("What is the revenue for 2023?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Meta First Quarter 2024 Results

Reconciliation of GAAP to Non-GAAP Results

(In millions, except percentages) (Unaudited) Three Months Ended March 31, 2024 2023
| |2024|2023|
|---|---|---|
|GAAP revenue|$36,455|$28,645|
|Foreign exchange effect on 2024 revenue using 2023 rates|($106)| |
|Revenue excluding foreign exchange effect|$36,349| |
|GAAP revenue year-over-year change %|27%| |
|Revenue excluding foreign exchange effect year-over-year change %|27%| |
|GAAP advertising revenue|$35,635|$28,101|
|Foreign exchange effect on 2024 advertising revenue using 2023 rates|($105)| |
|Advertising revenue excluding foreign exchange effect|$35,530| |
|GAAP advertising revenue year-over-year change %|27%| |
|Advertising

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
CPU times: user 5.84 s, sys: 399 ms, total: 6.24 s
Wall time: 3.49 s


In [31]:
print_response(response)

**Answer:** The revenue for 2023 is $28,645 million.

**Additional helpful information:**

* The revenue for 2024 is $36,455 million, which represents a 27% year-over-year increase.
* The majority of the revenue comes from advertising, with $35,635 million in 2024 and $28,101
million in 2023.
* The "Family of Apps" segment, which includes Facebook, Instagram, and WhatsApp, generated $36,015
million in revenue in 2024, while the "Reality Labs" segment generated $440 million in revenue in
2024.


In [32]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2024? Calculate the answer"
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: First Quarter 2024 Operational and Other Financial Highlights

Family daily active people (DAP) – DAP was 3.24 billion on average for March 2024, an increase of 7% year-over-year.

Ad impressions – Ad impressions delivered across our Family of Apps increased by 20% year-over-year.

Average price per ad – Average price per ad increased by 6% year-over-year.

Revenue – Total revenue and revenue on a constant currency basis were $36.46 billion and $36.35 billion, respectively, both of which increased by 27% year-over-year.

Costs and expenses – Total costs and expenses were $22.64 billion, an increase of 6% year-over-year.

Capital expenditures – Capital expenditures, including principal payments on finance leases

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
CPU times: user 6.15 s, sys: 325 ms, total: 6.48 s
Wall time: 5.29 s


In [33]:
print_response(response)

**Revenue minus Costs and Expenses for 2024:**

Based on the provided information, we can calculate the revenue minus costs and expenses for 2024 as
follows:

**Revenue:** Not explicitly stated for 2024, but we have a range for Q2 2024 ($36.5-39 billion) and
a growth rate of 27% year-over-year for Q1 2024. Let's assume an average revenue for 2024, which is
not possible to calculate exactly with the given information.

**Costs and Expenses:** Expected to be in the range of $96-99 billion for 2024.

Since we don't have an exact revenue figure for 2024, we cannot calculate the exact difference
between revenue and costs and expenses. However, we can say that the costs and expenses are expected
to be significantly higher than the revenue, indicating a potential loss for 2024.

**Additional Information:**

* The company has a strong cash position, with $58.12 billion in cash, cash equivalents, and
marketable securities as of March 31, 2024.
* The company is investing heavily in infrastructur

In [34]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2023? Calculate the answer"
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Meta First Quarter 2024 Results

Reconciliation of GAAP to Non-GAAP Results

(In millions, except percentages) (Unaudited) Three Months Ended March 31, 2024 2023
| |2024|2023|
|---|---|---|
|GAAP revenue|$36,455|$28,645|
|Foreign exchange effect on 2024 revenue using 2023 rates|($106)| |
|Revenue excluding foreign exchange effect|$36,349| |
|GAAP revenue year-over-year change %|27%| |
|Revenue excluding foreign exchange effect year-over-year change %|27%| |
|GAAP advertising revenue|$35,635|$28,101|
|Foreign exchange effect on 2024 advertising revenue using 2023 rates|($105)| |
|Advertising revenue excluding foreign exchange effect|$35,530| |
|GAAP advertising revenue year-over-year change %|27%| |
|Advertising

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
CPU times: user 4.02 s, sys: 302 ms, total: 4.32 s
Wall time: 3.47 s


In [35]:

print_response(response)

To calculate the revenue minus the costs and expenses for 2023, we can use the following
information:

Revenue: $28,645 million (from the "First Quarter 2024 Financial Highlights" section)
Costs and expenses: $21,418 million (from the "First Quarter 2024 Financial Highlights" section)

Revenue minus costs and expenses for 2023:
$28,645 million - $21,418 million = $7,227 million

This is also the income from operations for 2023, as shown in the "First Quarter 2024 Financial
Highlights" section.


In [36]:
%%time
response = qa.invoke("What is the expected revenue for the second quarter of 2024?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were $58.12 billion as of March 31, 2024. Free cash flow was $12.53 billion.

Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.

Meta First Quarter 2024 Results

CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growth, based on current exchange rates.

We expect full-year 2024 total expenses to be in the range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Rea

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
CPU times: user 5.82 s, sys: 336 ms, total: 6.15 s
Wall time: 5.69 s


In [37]:
Markdown(response["result"])

**Answer:** The expected revenue for the second quarter of 2024 is in the range of $36.5-39 billion.

**Additional information:** This guidance assumes a 1% headwind to year-over-year total revenue growth due to foreign currency exchange rates.